<a href="https://colab.research.google.com/github/suchitra2020180/RS_GIS_Python/blob/main/Basic_SAR_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Followed:https://www.youtube.com/watch?v=PiU68g3WRIY&t=1900s

In [ ]:
#RUS Webinar-Copernicus data processing in python using snappy-py01

In [ ]:
##   Run the code in Jupyter notebook

In [ ]:
#1. Load Python Modules
#Module                                                         #Desription
import numpy as np  #scientific computing
import pandas as pd  #Data analytics and manipulation
import subprocess    #External calls to the system
import snappy  #SNAP Python interface
import jpy   #Python-jAVA bridge
from glob import iglob #Data access in file manager
from os.path import join  #Data access in file manager
from zipfile import ZipFile  ##Zip file manipulation
from termcolor import coloured #prints colored text
import matplotlib.pyplot as plt #Create visualizations
import matplotlib.image as mpimg #Create visualizations
import matplotlib.colors as colors  #Create visualizations


ModuleNotFoundError: No module named 'snappy'

In [ ]:
#2. User-defined functions
# To facilitate the processing of data and avoid code duplications in this exercise, we will declare now user-defined functions which will be used later.

In [ ]:
def output_view(product, band, min_value_VV,max_value_VV,min_value_VH,max_value_VH):
  '''
  Creates a visualisation of the processed Sentinel-1 SAR image
  keyword arguements:
  produt          --snappy GPF product --> input Sentinel-1 product
  band            --List --> product's band to be visualized
  min_value_VV    --int --> min value for color stretch in VV band
  max_value_VV    --int --> max value for color stretch in VV band
  min_value_VH    --int --> min value for color stretch in VH band
  max_value_VH    --int --> max value for color stretch in VH band
  '''
  band_data_list=[]
  for i in band:
    band=product.getBand(i)
    w=band.getRasterWidth()
    h=band.getRasterHeight()
    band_data=np.zeros(w*h, np.float32)
    band.readPixels(0,0,w,h,band_data)
    band_data.shape=h,w
    band_data_list.append(band_data)

  fig, (ax1,ax2) =plt.subplots(1,2,figsize=(16,16))
  ax1.imshow(band_data_list[0], cmap='gray', vmin=min_value_VV, vmax=max_value_VV)
  ax1.set_title(output_bands[0])
  ax2.imshow(band_data_list[1], cmap='gray', vmin=min_value_VH, vmax=max_value_VH)
  ax2.set_title(output_bands[1])

  for ax in fig.get_axes():
    ax.label_outer()


In [ ]:
# 3. Sentinel-1 Preprocessing: snappy
#Easiest way to know which operators are available in snappy is to cal   "gpt"  "-h" from command line, which will output the list.If we want to access the documentation of a specific operator, use gpt -h *Operator*

In [ ]:
# Call gpt -h from command line
print(subprocess.Popen(['gpt','-h','ThermalNoiseRemoval'], stdout=subprocess.PIPE,universal_newlines=True).communicate()[0])

In [ ]:
#3.1 Read Product

In [ ]:
#Set target folder and extract metadata
produt_path="/shared/Training/PY01_SentinelProcessing_snappy/Original/"
input_S1_files=sorted(list(iglob(join(product_path, '**','*S1*.zip'),recursive=True)))

name, sensing_mode, product_type, polarization, height, width, band_names =([] for i in rane(7))

for i in input_S1_files:
  sensing_mode.append(i.split("_")[3])
  product_type.append(i.split("_")[4])
  polarization.append(i.split("_")[-6])
  #Read with snappy
  s1_read=snappy.ProductIO.readProduct(i)
  name.append(s1_read.getName())
  height.append(s1_read.getSceneRasterHeight())
  width.append(s1_read.getSceneRasterWidth())
  band_names.append(s1_read.getBandNames())

df_s1_read=pd.DataFrame({'Name':name,'Sensing Mode':sensing_mode,'Product Type':product_type, 'Polarization':polarization, 'Height':height, 'Width':width})
dispaly(df_s1_read)

#Display quicklook. First image
with ZipFile(input_S1_Files[0],'r') as qck_look:
  qck_look=qck_look.open(name[0] + '.SAFE/preview/quick-look.png')
  img=mpimg.imread(qck_look)
  plt.figure(figsize=(15,15))
  plt.title('Quicklook Visualization -'+ name[0] +'\n')
  plt.axis('off')
  plt.imshow(img);


In [ ]:
#3.2 Subset

In [ ]:
# ur corner
x,y, width, height =12000, 8000, 5500, 5500
#Subset Operator -snappy
parameters =snappy.HashMap()
parameters.put('copyMetadata',True)
parameters.put('region',"%s,%s,%s,%s" % (x, y, width, height))
subset=snappy.GPF.createProduct('Subset',parameters, s1_read)
print(list(subset.getBandNames()))

#Plot subset(follow VV- VH order)
output_bands=['Amplitude_VV', 'Amplitude_VH']
output_view(subset, output_bands,41, 286, 20, 160)


In [ ]:
#3.3 Apply Orbit File

In [ ]:
#The orbit state vectors provided in the metadata of a SAR product are generally not accurate and can be refined with the precise orbit files which are available days-to-weeks after the generation of the product.


SyntaxError: invalid syntax (<ipython-input-1-7cac16a0aa3d>, line 1)

In [ ]:
#The orbit file provides accurate satellite position and velocity information.Based on the information, the orbit state vectors in the

In [ ]:
#Apply Orbit File Operator -snappy
parameters=snappy.HashMap()
parameters.put('Apply-Orbit-File', True)
apply_orbit=snappy.GPF.createProduct('Apply-Orbit-File', parameters, subset)
print(colored('Orbit updates successfully','green'))


In [ ]:
#3.4 Thermal Noise Removal

In [ ]:
#Thermal Noise Removal Operator -snappy
parameters=snappy.HashMap()
parameters.put('removeThermalNoise', True)
thermal_noise=snappy.GPF.createProduct('ThermalNoiseRemoval', parameters, apply_orbit)

#Plot Thermal Noise Removal (follow VV -VH order)
output_bands=['Intensity_VV','Intensity_VH']
output_view(thermal_noise, output_bands, 0.02,99376.52,0.27,18371.83)


In [ ]:
#3.5 Radiometric Calibration

The objective of SAR calibration is to provide imagery in which the pixel values can be directly related to the radar backscattering of the scaene.Though uncalibrated SAR imagery is sufficient for qualitative use, calibrated SAR images are essential to quantitative use of SAR data. Typically SAR data processing, which produces level-1 images, doe snot include radiometric correction and significant radiometric bias remains. The radiometric correction necessary for the pixel values to truely represent the radar backscatter of the reflecting surface and therefore for comparison of SAR images acquired with different sensors, or acquired from the same sensor but at different times, in different modes, or processed by different processors.

In [ ]:
#Calibration Operator -snappy
parameters = snappy.HashMap()
parameters.put('outputSigmaBand',True)
parameters.put('sourceBands','Intensity_VH,Intensity_VV')
parameters.put('selectedPolarisations','VH,VV')
parameters.put('outputImageScaleInDb',False)
calibrated=snappy.GPF.createProduct('Calibration',parameters, thermal_noise)

#Plot Calibration (follow VV-VH order)
output_bands=['Sigma0_VV','Sigma0_VH']
output_view(calibrated, output_bands, 0.00, 0.28, 0.00, 0.05)

In [ ]:
#To get the list of parameters required for calibration, we can use the following command:

In [ ]:
#3.6 Speckle Filtering

SAR images have inherent salt and pepper like txturing called speckle which degrades the quality of the image and make interpretation of features more difficult. Speckles are caused by random constructive nd destructive interference of the de-phased but coherent retuen waves scattered by the elementary sctters within each resolution cell. Speckle noise reduction can be applied either by spatial filtering or multilook processing.

In [ ]:
# Speckle Filtering Operator -snappy
parameters =snappy.HashMap()
parameters.put('filter','Lee')
parameters.put('filterSizeX',5)
parameters.put('filterSizeY',5)
speckle=snappy.GPF.createProduct('speckle-Filter', parameters, calibrated)

#Plot speckle filter(follow VV-VH order)
output_bands=['Sigma0_VV','Sigma0_VH']
output_view(speckle, output_bands, 0.00,0.21,0.00,0.048)

In [ ]:
#3.7 Terrain Correction

Due to the topographical varaiations in the scene and the tilt of the satellite sensor, distances can be distorted in the SAR images. Image data not directly at the sensor's Nadir location will have some distortion. Terrain correction are intended to compensate for these distortions so that the geometric representation of the image will be as clase as possible to the real world.

In [ ]:
proj='''PROJCS["UTM Zone 4 / World Geodetic System 1984",
  GEOGCS["World Geodetic System 1984",
  DATUM["World Geodetic System 1984",
  SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]],
  AUTHORITY["EPSG","6326"]],
  PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]],
  UNIT["degree", 0.017453292519943295],
  AXIS["Geodetic longitude", EAST],
  AXIS["Geodetic latitude", NORTH]],
  PROJECTION["Transverse_Mercator"],
  PARAMETER["central_meridian", 3.0],
  PARAMETER["latitude_of_origin", 0.0],
  PARAMETER["scale_factor", 0.9996],
  PARAMETER["false_easting", 500000.0],
  PARAMETER["false_northing", 0.0],
  UNIT["m", 1.0],
  AXIS["Easting", EAST],
  AXIS["Northing", NORTH]]'''

#Terrain-Correction Operator -snappy
parameters=snappy.HashMap()
parameters.put('demName','SRTM 3Sec')
parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
parameters.put('pixelSpacingInMeter',10.0)
parameters.put('mapProjection', proj)       # comment this line if no need to convert to UTM/WGS84, default is WGS84
parameters.put('nodataValueAtSea', False)  #do not mask areas without elevation
parameters.put('saveProjectedLocalIncidenceAngle', True)
parameters.put('saveSelectedSourceBand', True)
terrain_correCtion=snappy.GPF.createProduct('Terrain-Correction', parameters, speckle)

# Plot terrain correction (follow VV-VH order)
output_bands=['Sigma0_VV', 'Sigma0_VH'] # in this step Amplitude bands are lost?
output_view(terrain_correction, output_bands, 0.00, 0.49, 0.00, 0.04)

In [ ]:
#UTM projection parameters #github
proj='''PROJCS["UTM Zone 4 / World Geodetic System 1984",
  GEOGCS["World Geodetic System 1984",
  DATUM["World Geodetic System 1984",
  SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]],
  AUTHORITY["EPSG","6326"]],
  PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]],
  UNIT["degree", 0.017453292519943295],
  AXIS["Geodetic longitude", EAST],
  AXIS["Geodetic latitude", NORTH]],
  PROJECTION["Transverse_Mercator"],
  PARAMETER["central_meridian", -159.0],
  PARAMETER["latitude_of_origin", 0.0],
  PARAMETER["scale_factor", 0.9996],
  PARAMETER["false_easting", 500000.0],
  PARAMETER["false_northing", 0.0],
  UNIT["m", 1.0],
  AXIS["Easting", EAST],
  AXIS["Northing", NORTH]]'''




In [ ]:
#3.8 Write

Once we have completed all preprocessing steps we can write our SAR products to file. In this occasion we will choose the GeoTiff format

In [ ]:
#Set output path and name
outpath_name='/shared/Training/PY01_Sentinel1Processing_snappy/Processing/snappy/20200418_Orb_Ther_Cal_Spk_TC'

#Write Operator -snappy
snappy.ProductIO.writeProduct(terrain_correction, outpath_name,'GeoTIFF')
print(colored('Product successfully saved in:', 'green'), outpath_name)